In [1]:
import torch
import torch.nn as nn
import numpy as np
import pickle as pk
from model import GlobalPredictor

In [2]:
data = []
for d in range(15, 22):
    filename = '/data/fan/UsersInOsakaProcessed/201210{:02d}_interp.pk'.format(d)
    print(filename)
    with open(filename, 'rb') as f:
        data += list(pk.load(f).values())
data = np.stack(data, axis=0)



/data/fan/UsersInOsakaProcessed/20121015_interp.pk


FileNotFoundError: [Errno 2] No such file or directory: '/data/fan/UsersInOsakaProcessed/20121015_interp.pk'

In [ ]:
val_data = []
for d in range(22, 29):
    filename = '/data/fan/UsersInOsakaProcessed/201210{:02d}_interp.pk'.format(d)
    print(filename)
    with open(filename, 'rb') as f:
        val_data += list(pk.load(f).values())
val_data = np.stack(val_data, axis=0)

In [ ]:
data.shape

In [ ]:
data = torch.LongTensor(data).cuda(0)
val_data = torch.LongTensor(val_data).cuda(0)

In [ ]:
num_locs = 1600
loc_embedding_dim = 128
T = 96
num_time = T
time_embedding_dim = 32
hidden_dim = 256
latent_dim = 256

In [ ]:
global_predictor = GlobalPredictor(num_locs, loc_embedding_dim, num_time, time_embedding_dim, hidden_dim, latent_dim).cuda(0)

In [ ]:
optimizer = torch.optim.SGD(global_predictor.parameters(), lr=1e-3, momentum=0.8)
optimizer_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [ ]:
dT = 4

In [ ]:
training_loss = dict({})
validation_loss = dict({})

In [ ]:
batch_size = 128

for epoch in range(1, 21):
    optimizer.zero_grad()
    optimizer_scheduler.step()
    perm = torch.randperm(data.shape[0])
    #返回一个0到n-1的数组
    #data.shape = (250770, 96)
    #data.shape[0] 
    
    
    
    avg_loss = 0.0
    
    cnt = 0
    for i in range(0, data.shape[0], batch_size):
        data_batch = data[perm[i: i + batch_size]]
        t = np.random.randint(T - 2 * dT + 1)
        # 在这个区间范围内挑选一个值                
        time_batch = torch.zeros_like(data_batch[:, t: t + dT]) + t
        loss = global_predictor(data_batch[:, t: t + dT], time_batch, data_batch[:, t + 2 * dT - 1])
        
        
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        avg_loss += loss.item()
        print('Epoch {:02d}, avg_loss = {:.4f}'.format(epoch, avg_loss / (i + batch_size)), end='\r')
    training_loss[epoch] = avg_loss / data.shape[0]
    print('')
    
    
    
    
    
    
    
    # testing
    avg_loss = 0.0
    perm = torch.randperm(val_data.shape[0])
    with torch.no_grad():
        for i in range(0, val_data.shape[0], batch_size):
            data_batch = val_data[perm[i: i + batch_size]]
            t = np.random.randint(T - 2 * dT + 1)
            time_batch = torch.zeros_like(data_batch[:, t: t + dT]) + t
            loss = global_predictor(data_batch[:, t: t + dT], time_batch, data_batch[:, t + 2 * dT - 1])
            avg_loss += loss.item()
    print('Validation Loss = {:.4f}'.format(avg_loss / val_data.shape[0]))
    validation_loss[epoch] = avg_loss / val_data.shape[0]
    

In [ ]:
torch.save(global_predictor, './results_osaka/global_predictor.pytorch')

In [ ]:
with open('./results_osaka/centralized_global_predictor_training_loss.pk', 'wb') as f:
    pk.dump(training_loss, f)
with open('./results_osaka/centralized_global_predictor_validation_loss.pk', 'wb') as f:
    pk.dump(validation_loss, f)